In [1]:
pip install --force-reinstall -v nltk==3.6.1

Using pip 22.3.1 from /opt/conda/lib/python3.7/site-packages/pip (python 3.7)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.1/757.1 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: zipp
    Found existing installation: zipp 3.11.0
    Uninstalling zipp-3.11.0:
      Removing file or directory /opt/conda/lib/python3.7/site-packages/zipp-3.11.0.dist-info/INSTALLER
      Removing file or directory /opt/conda/lib/python3.7/site-packages/zipp-3.11.0.dist-info/LICENSE
      Removing file or directory /opt/conda/lib/python3.7/site-packages/zipp-3.11.0.dist-info/METADATA
      Removing file or directory /opt/conda/lib/python3.7/site-packages/zipp-3.11.0.dist-info/RECOR

In [2]:
import json
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import ngrams
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train = pd.read_json(path_or_buf="/kaggle/input/vc-it-cup-ranking/train_preprocessed.json")
test = pd.read_json(path_or_buf="/kaggle/input/vc-it-cup-ranking/test_preprocessed.json")

In [4]:
train.head()

,comments,score,posts
0,[go back school ident give founder go back sch...,"[0, 1, 2, 3, 4]",mani summer combin funde decid continu startup...
1,[curious head long run cbs tear fit imag tri e...,"[0, 1, 2, 3, 4]",cbs acquir lastfm 280m
2,[hate fals claim fiduciari respons public comp...,"[0, 1, 2, 3, 4]",costco becam antiwalmart
3,[real point simpl power order sound edgi subve...,"[0, 1, 2, 3, 4]",fortun favor big turd screw money art
4,"[look someon hasnt read mythic manmonth, chanc...","[0, 1, 2, 3, 4]",startupweekend 70 founder creat one compani we...


In [5]:
def comments_len(df):
    '''
    Функция для опредления длины коммента
    '''
    df_len = df.apply(lambda x: x.str.len(), axis=1)
    df_len['text'] = df['text']
    return df_len

In [6]:
train_score = train['comments'].apply(pd.Series)
train_score['text'] = train['posts']

In [25]:
test_score = test['comments'].apply(pd.Series)
test_score['text'] = test['posts']

In [7]:
train_score.head()

,0,1,2,3,4,text
0,go back school ident give founder go back scho...,invari success set fall back origin path that ...,school connect go real world enter school thin...,guess depend hungri believ product im 24 still...,pollground decid go back school get combin fund,mani summer combin funde decid continu startup...
1,curious head long run cbs tear fit imag tri es...,mean there bignam compani fight repeal recent ...,also bbc news httpnewsbbccouk1lowtechnology670...,understand worth 70m year,sold cheapli given leadership posit ask least ...,cbs acquir lastfm 280m
2,hate fals claim fiduciari respons public compa...,love costco best quot articl thisth tradit ret...,mr sineg warn costco increas markup 16 18 perc...,costco treat employe littl bit better normal i...,id advertis strategi upscal shopper talk frien...,costco becam antiwalmart
3,real point simpl power order sound edgi subver...,im think well ive never heard bug tracker call...,ive never seen singl blog post wander mani dir...,rant person blog vent frustrat vapid diggbait ...,etern word geico cavemanwhat,fortun favor big turd screw money art
4,look someon hasnt read mythic manmonth,chanc recreat telecommut love attend cross atl...,start 70 elimin three four realityshow style t...,weekendid love hear happen nyc nyc startup hub,interest join event happen sv equiti fun,startupweekend 70 founder creat one compani we...


# Длинны коментариев

In [8]:
df = comments_len(train_score)

In [9]:
df.head()

,0,1,2,3,4,text
0,520,176,256,150,47,mani summer combin funde decid continu startup...
1,142,71,83,25,50,cbs acquir lastfm 280m
2,176,100,219,198,70,costco becam antiwalmart
3,1513,343,319,68,28,fortun favor big turd screw money art
4,38,258,73,46,40,startupweekend 70 founder creat one compani we...


In [10]:
df[df.loc[:, 0] == df.loc[:, 0:4].apply(max, axis=1)].shape

(40785, 6)

In [11]:
df[df.loc[:, 1] == df.loc[:, 0:4].apply(max, axis=1)].shape

(19951, 6)

In [12]:
df[df.loc[:, 2] == df.loc[:, 0:4].apply(max, axis=1)].shape

(12391, 6)

In [13]:
df[df.loc[:, 3] == df.loc[:, 0:4].apply(max, axis=1)].shape

(8827, 6)

In [14]:
df[df.loc[:, 4] == df.loc[:, 0:4].apply(max, axis=1)].shape

(6411, 6)

# Расстояние Левенштейна(не используется)

In [15]:
from Levenshtein import distance as lev

def levenshtein_dist(df):
    '''
    Расстояние Левенштейна между постом и комментариями
    '''
    df_lev = pd.DataFrame({'text': df.text})
    for i in range(5):
        df_lev[i] = df.apply(lambda x: lev(x.text, x[i]), axis=1)
    
    return df_lev

In [16]:
levenshtein_dist_df = levenshtein_dist(train_score)

In [17]:
levenshtein_dist_df.to_csv('levenshein_distance.csv')

In [18]:
print(nltk.__version__)

3.6.1


# Расстояние Яро-Винклера

In [19]:
from nltk.metrics import distance

def jaro_winkler_dist(df):
    '''
    Расстояние Яро-Винклера между постом и комментариями
    '''
    df_j_w = pd.DataFrame({'text': df.text})
    for i in range(5):
        df_j_w[i] = df.apply(lambda x: distance.jaro_winkler_similarity(x.text, x[i]), axis=1)
    
    return df_j_w

In [21]:
jaro_winkler_distances = jaro_winkler_dist(train_score)

In [24]:
jaro_winkler_distances.to_csv('jaro_winkler_distances_train.csv')

In [26]:
jaro_winkler_distances_test = jaro_winkler_dist(test_score)

In [27]:
jaro_winkler_distances_test.to_csv("jaro_winkler_distances_test.csv")

# Сентимент

In [28]:
reviews_train = pd.read_csv('/kaggle/input/vc-it-cup-ranking/train_data_twitter.csv', index_col=0)

In [29]:
reviews_train

,id,information,type,text,lower
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting on borderlands 2 and i will murder ...
...,...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...,just realized that the windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...,just realized that my mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...,just realized the windows partition of my mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...,just realized between the windows partition of...


In [30]:
stopwords_nltk = nltk.corpus.stopwords
stop_words = stopwords_nltk.words('english')
stop_words[:5]

['i', 'me', 'my', 'myself', 'we']

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

#Initial Bag of Words
bow_counts = CountVectorizer(
    tokenizer=word_tokenize,
    stop_words=stop_words, #English Stopwords
    ngram_range=(1, 1) #analysis of one word
)

In [32]:
X_train_bow = bow_counts.fit_transform(reviews_train.lower.astype('U'))

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  % sorted(inconsistent)


In [33]:
y_train_bow = reviews_train['type']

In [36]:
from sklearn.linear_model import LogisticRegression

# Logistic regression
model1 = LogisticRegression(C=1, solver="liblinear",max_iter=200)
model1.fit(X_train_bow, y_train_bow)
# # Prediction
# test_pred = model1.predict(X_test_bow)
# print("Accuracy: ", accuracy_score(y_test_bow, test_pred) * 100)

LogisticRegression(C=1, max_iter=200, solver='liblinear')

In [46]:
train_sent_df = pd.DataFrame()

In [47]:
from tqdm import trange

for i in trange(5):
    train_pr_bow = bow_counts.transform(train_score.loc[:, i])
    train_pred = model1.predict(train_pr_bow)
    train_sent_df[i] = train_pred

100%|██████████| 5/5 [02:20<00:00, 28.08s/it]


In [49]:
train_sent_df.to_csv('train_sent_df.csv')

In [51]:
for i in range(5):
    print(f"Количество позитивных и негативных комментариев для {i} столбца: {(train_sent_df[train_sent_df.loc[:, i] == 'Positive'].count()[0], train_sent_df[train_sent_df.loc[:, i] == 'Negative'].count()[0])}")

Количество позитивных и негативных комментариев для 0 столбца: (10575, 21756)
Количество позитивных и негативных комментариев для 1 столбца: (13556, 22822)
Количество позитивных и негативных комментариев для 2 столбца: (15590, 23650)
Количество позитивных и негативных комментариев для 3 столбца: (17284, 23880)
Количество позитивных и негативных комментариев для 4 столбца: (19273, 24419)


In [52]:
test_sent_df = pd.DataFrame()

In [53]:
from tqdm import trange

for i in trange(5):
    test_pr_bow = bow_counts.transform(test_score.loc[:, i])
    test_pred = model1.predict(test_pr_bow)
    test_sent_df[i] = test_pred

100%|██████████| 5/5 [00:23<00:00,  4.71s/it]


In [54]:
test_sent_df.to_csv('test_sent_df.csv')